<a href="https://colab.research.google.com/github/illliyas/Machine-Learning/blob/master/tweets/Topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import matplotlib.pyplot as plt
from math import log, sqrt
import pandas as pd
import numpy as np
import re
import spacy
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-t

In [2]:
!python -m spacy download en_core_web_sm
!pip install locationtagger

2022-11-05 12:53:14.814844: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 14.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 14.7 MB/s 
     |████████████████████████████████| 211 kB 60.5 MB/s 
     |████████████████████████████████| 10.1 MB 46.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81 kB 7.7 MB/s 
     |████████████████████████████████| 7.4 MB 44.4 MB/s 
     |████████████████████████████████| 93 kB 2.3 MB/s 
  Created

In [3]:
import locationtagger

In [4]:
df = pd.read_csv('ek_tweets4.csv')

In [5]:
def cleaning_tweets(tweet):
    temp = tweet.lower()
    #Removing hashtags and mentions
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    #Removing links
    temp = re.sub(r"http\S+", "", temp)
    temp = re.sub(r"www.\S+", "", temp)
    #Removing punctuations
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    #Filtering non-alphanumeric characters
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = re.sub("([a-z]*[0-9]+[a-z0-9]* )+"," ", temp)
    #remove 2 letter words
    temp = re.sub(r"\b[a-z]{2}\b"," ", temp)
    return temp
sw = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
def remove_stopwords(tweet):
    words = word_tokenize(tweet)
    words = [word for word in words if word not in sw]
    words = [lemmatizer.lemmatize(word) for word in words]
    #words = [stemmer.stem(word) for word in words]
    return ' '.join(words) 
def get_places(tweet):
    place_entity = locationtagger.find_locations(text = tweet)
    places_list = [y for x in [ place_entity.countries + place_entity.regions + place_entity.cities] for y in x]
    places_list = [x.lower() for x in places_list]
    places_list = list(dict.fromkeys(places_list))
    return ' '.join(places_list)
def remove_places(tweet):
    place_entity = locationtagger.find_locations(text = tweet)
    places_list = [y for x in [ place_entity.countries + place_entity.regions + place_entity.cities] for y in x]
    places_list = [x.lower() for x in places_list]
    places_list = list(dict.fromkeys(places_list))
    words = word_tokenize(tweet)
    words = [i for i in words if i not in places_list]
    return ' '.join(words)
def number_of_words(tweet):
  return len(word_tokenize(tweet))    

In [6]:
df['tweet_clean'] = df.apply(lambda row: cleaning_tweets(row['tweet']),axis=1)
df['tweet_clean2'] = df.apply(lambda row: remove_stopwords(row['tweet_clean']),axis=1)
df['tweet_places'] = df.apply(lambda row: get_places(row['tweet_clean2']),axis=1)
df['tweet_clean3'] = df.apply(lambda row: remove_places(row['tweet_clean2']),axis=1)
df['number_of_words'] = df.apply(lambda row: number_of_words(row['tweet_clean3']),axis=1)

In [7]:
list(df['tweet'])[:5]

['@EmiratesSupport Thanks for the support, hope it will get resolved asap :)',
 '@EmiratesSupport Have replied to message. Please check',
 "@EmiratesSupport @EmiratesSupport i have DM'd u. but no one seems to reply.  can u please help in this regard?",
 '@EmiratesSupport sent you a dm Janine',
 '@EmiratesSupport I have been mailing for flight delay compensation but no one is responding till now. Please let me know  when can I get a proper reply.']

In [8]:
list(df['tweet_clean3'])[:5]

['thanks support hope get resolved asap',
 'replied message please check',
 'u one seems reply u please help regard',
 'sent janine',
 'mailing flight delay compensation one responding till please let know get proper reply']

In [9]:
df2 = df[df['number_of_words']>2]

In [10]:
df2=df2.reset_index()

In [11]:
df2.to_csv('ek_tweets_clean.csv',index=False)

In [ ]:
---------------------

In [2]:
!pip install --upgrade joblib==1.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
pip install joblib==1.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from bertopic import BERTopic

In [5]:
df2 = pd.read_csv('ek_tweets_clean.csv')

In [6]:
df2['created_at'] = pd.to_datetime(df2['created_at'])

In [7]:
tweets = df2.tweet_clean2.to_list()
timestamps  = df2.created_at.to_list()

In [12]:
model = BERTopic(verbose=True,embedding_model='paraphrase-MiniLM-L3-v2', min_topic_size= 6)
topics, _ = model.fit_transform(tweets)

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

2022-11-05 13:37:11,697 - BERTopic - Transformed documents to Embeddings
2022-11-05 13:37:15,817 - BERTopic - Reduced dimensionality
2022-11-05 13:37:15,854 - BERTopic - Clustered reduced embeddings


In [ ]:
model = BERTopic(verbose=True,embedding_model='all-mpnet-base-v2', min_topic_size= 3)
topics, _ = model.fit_transform(tweets)

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

2022-10-23 19:52:47,568 - BERTopic - Transformed documents to Embeddings
2022-10-23 19:52:51,252 - BERTopic - Reduced dimensionality
2022-10-23 19:52:51,292 - BERTopic - Clustered reduced embeddings


In [13]:
model.get_topic_info()

,Topic,Count,Name
0,-1,131,-1_flight_change_hour_service
1,0,110,0_dubai_luggage_baggage_emirate
2,1,61,1_please_reply_thank_thanks
3,2,60,2_complaint_email_customer_form
4,3,35,3_booking_ticket_booked_showing
5,4,22,4_flight_refund_rebook_cancelled
6,5,20,5_emirate_ground_staff_support
7,6,18,6_test_vaccinated_vaccination_covid
8,7,14,7_seat_dxb_selection_get
9,8,12,8_nut_meal_offer_cater


In [15]:
model.reduce_topics(tweets, nr_topics=10)

2022-11-05 13:38:03,123 - BERTopic - Reduced number of topics from 17 to 11


In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549 entries, 0 to 548
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   index            549 non-null    int64              
 1   author id        549 non-null    float64            
 2   created_at       549 non-null    datetime64[ns, UTC]
 3   geo              549 non-null    object             
 4   id               549 non-null    float64            
 5   lang             549 non-null    object             
 6   like_count       549 non-null    int64              
 7   quote_count      549 non-null    int64              
 8   reply_count      549 non-null    int64              
 9   retweet_count    549 non-null    int64              
 10  source           549 non-null    object             
 11  tweet            549 non-null    object             
 12  tweet_clean      549 non-null    object             
 13  tweet_clean2     549

,index,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,tweet_clean,tweet_clean2,tweet_places,tweet_clean3,number_of_words
162,179,63525135.0,2022-10-13 14:19:36+00:00,,1.580000e+18,en,290,0,19,9,Twitter for iPhone,@EmiratesSupport just tried to check in for ou...,just tried check for our flight saturday ...,tried check flight saturday system decided yea...,NaN,tried check flight saturday system decided yea...,14
169,186,63525135.0,2022-10-13 16:26:42+00:00,,1.580000e+18,en,10,0,2,1,TweetDeck,@EmiratesSupport Thank you so much for resolvi...,thank you much for resolving the issue muc...,thank much resolving issue much appreciated,NaN,thank much resolving issue much appreciated,6


In [25]:
df2['tweet_topics'] = topics

In [26]:
df2[df2['author id']==63525135]

,index,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,tweet_clean,tweet_clean2,tweet_places,tweet_clean3,number_of_words,tweet_topics
162,179,63525135.0,2022-10-13 14:19:36+00:00,,1.580000e+18,en,290,0,19,9,Twitter for iPhone,@EmiratesSupport just tried to check in for ou...,just tried check for our flight saturday ...,tried check flight saturday system decided yea...,NaN,tried check flight saturday system decided yea...,14,9
169,186,63525135.0,2022-10-13 16:26:42+00:00,,1.580000e+18,en,10,0,2,1,TweetDeck,@EmiratesSupport Thank you so much for resolvi...,thank you much for resolving the issue muc...,thank much resolving issue much appreciated,NaN,thank much resolving issue much appreciated,6,1


In [30]:
model.topic_labels_

{-1: '-1_flight_help_change_please',
 0: '0_dubai_luggage_emirate_baggage',
 1: '1_please_reply_thank_thanks',
 2: '2_complaint_email_customer_form',
 3: '3_booking_ticket_booked_showing',
 4: '4_emirate_airline_issue_sorry',
 5: '5_flight_refund_rebook_cancelled',
 6: '6_test_vaccinated_vaccination_covid',
 7: '7_seat_dxb_get_selection',
 8: '8_nut_meal_offer_cater',
 9: '9_saturday_system_decided_shoul'}

In [13]:
model.visualize_topics()

In [14]:
model.visualize_barchart()

In [33]:
topics_over_time = model.topics_over_time(tweets, timestamps)

530it [00:04, 111.98it/s]


In [35]:
model.visualize_topics_over_time(topics_over_time, topics=[0, 1, 2, 3])

In [36]:
from sklearn.datasets import fetch_20newsgroups

# Prepare data and classes
data = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))

In [38]:
docs = data["data"]